In [1]:
from __future__ import print_function
import pickle
import os.path
import base64
import email
from apiclient import errors
import re
import time
from datetime import datetime
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    
    return service

#     # Call the Gmail API
#     results = service.users().labels().list(userId='me').execute()
#     labels = results.get('labels', [])
    
#     userInfo = service.users().getProfile(userId='me').execute()
#     print("User Info :",userInfo)

#     if not labels:
#         print('No labels found.')
#     else:
#         print('Labels:')
#         for label in labels:
#             print(label['name'])





# if __name__ == '__main__':
#     main()

main()

In [2]:
results=main().users().labels().list(userId='me').execute()
label=results.get('labels', [])
[i['name'] for i in label]

['CHAT',
 'SENT',
 'INBOX',
 'IMPORTANT',
 'TRASH',
 'DRAFT',
 'SPAM',
 'CATEGORY_FORUMS',
 'CATEGORY_UPDATES',
 'CATEGORY_PERSONAL',
 'CATEGORY_PROMOTIONS',
 'CATEGORY_SOCIAL',
 'STARRED',
 'UNREAD']

In [3]:
results = main().users().messages().list(userId='me',labelIds = ['SPAM'],maxResults=500).execute()

In [4]:
messages = results.get('messages', [])

In [5]:
# results = main().users().messages().list(userId='me',labelIds = ['INBOX'],maxResults=500).execute
# messages = results.get('messages', [])

while 'nextPageToken' in results:
    page_token=results['nextPageToken']
#     print(page_token)
    results = main().users().messages().list(userId='me',labelIds = ['SPAM'],maxResults=500,pageToken=page_token).execute()
    messages.extend(results['messages'])
    

In [6]:
len([i['id'] for i in messages])

249

In [22]:
start=time.time()
print(start)
string_check= re.compile('[@_!#$%^&*()<>?/\|}{~]')
msg_read=[]
date=[]
ff=[]
sub=[]
text=[]
length=[]
id=[]
time_taken=[]
for i in range(len([i['id'] for i in messages])):
    if i%200==0:
        print((datetime.now()).strftime('%H:%M:%S'))
        time_taken.append((datetime.now()).strftime('%H:%M:%S'))
    print(i)
    print(messages[i]['id'])
    id.append(messages[i]['id'])
    msg=main().users().messages().get(userId='me',id=messages[i]['id'],format="full").execute()
    subject=[i['value'] for i in msg['payload']['headers'] if i['name']=='Subject' or i['name']=='SUBJECT'][0]
    From=[i['value'] for i in msg['payload']['headers'] if i['name']=='from' or i['name']=='FROM' or i['name']=='From'][0]
    Date=[i['value'] for i in msg['payload']['headers'] if i['name']=='Date' or i['name']=='DATE'][0]
    msg_read.append(msg['snippet'])
    date.append(Date)
    ff.append(From)
    sub.append(subject)
    msg=main().users().messages().get(userId='me',id=messages[i]['id'],format="raw").execute()
    try:
        msg_str=base64.urlsafe_b64decode(msg['raw']).decode()
    except:
        msg_str=base64.urlsafe_b64decode(msg['raw']).decode(encoding='windows-1252')
    mime_msg=email.message_from_string(msg_str)
    
    length.append(len(mime_msg.get_payload()))
    
    
    a=[]
    a1=[]
    if mime_msg.is_multipart():
        
        for payload in mime_msg.get_payload():
            a.append(payload.get_payload())
            
            while type(a[0])==list:
                a=[i.get_payload() for i in a[0]]
            
            
#         if type(a1[0])==list:
#             b=mime_msg.get_payload()[0]
            
#             for i in b.get_payload():
#                 a.append(i.get_payload())
                
#         else:
#             for payload in mime_msg.get_payload():
#                 a.append(payload.get_payload())
            
        
# #         for payload in mime_msg.get_payload():
# #             a.append(payload.get_payload())
    else:
        a.append(mime_msg.get_payload())
    
    t1=a[0].replace("\r"," ").replace("\n"," ")
    text.append(" ".join([i for i in t1.split() if (string_check.search(i)==None) and ('http' not in i)]))
    
    print("Subject: ",subject)
#     print("From: ",From)
#     print("Date: ",Date)
#     print(msg['snippet'])
    print("      Next        ")
    
print("time taken",time.time()-start) 
taken=time.time()-start

1587923538.1032784
23:22:18
0
171b70da99da5923
Subject:  UC Riverside School of Business Virtual Info Sessions
      Next        
1
171b4e3ad02eb734
Subject:  Your path to greatness
      Next        
2
171b4ba361440057
Subject:  MNC Job Oriented Course ▷▷Data Analyst ▷▷Free Python/Tableau/SAS Course  ▷Business Analytics  ▷▷ WFA-MIS Analyst ▷▷ Banking Analyst ▷ Finance Analyst ▷▷ HR Analyst  ▷▷SAP MNC Jobs ▷Online/Offline Course
      Next        
3
171b4b75fb01bf35
Subject:  MNC Job Oriented Course ▷▷Data Analyst ▷▷Free Python/Tableau/SAS Course  ▷Business Analytics  ▷▷ WFA-MIS Analyst ▷▷ Banking Analyst ▷ Finance Analyst ▷▷ HR Analyst  ▷▷SAP MNC Jobs ▷Online/Offline Course
      Next        
4
171b4b02015f93ed
Subject:  MNC Job Oriented Course ▷▷Data Analyst ▷▷Free Python/Tableau/SAS Course  ▷Business Analytics  ▷▷ WFA-MIS Analyst ▷▷ Banking Analyst ▷ Finance Analyst ▷▷ HR Analyst  ▷▷SAP MNC Jobs ▷Online/Offline Course
      Next        
5
171b45718c78cc6d
Subject:  [Free Training] -

In [ ]:
    len(time_taken)

In [ ]:
len(date)

In [ ]:
len(msg_read)

In [ ]:
msg_read[2]

In [ ]:
msg_read=[]
date=[]
ff=[]
for i in range(5):
    msg=main().users().messages().get(userId='me',id=messages[i]['id'],format="full").execute()
    subject=[i['value'] for i in msg['payload']['headers'] if i['name']=='Subject'][0]
    From=[i['value'] for i in msg['payload']['headers'] if i['name']=='From'][0]
    Date=[i['value'] for i in msg['payload']['headers'] if i['name']=='Date'][0]
    msg_read.append(msg['snippet'])
    date.append(Date)
    ff.append(From)
#     sub.append(subject)
    print(i+1)
    print("Subject: ",subject)
    print("From: ",From)
    print("Date: ",Date)
    print(msg['snippet'])
    print("              ")

In [ ]:
list(messages)[276]

In [ ]:
list(messages)[471]

In [ ]:
msg=main().users().messages().get(userId='me',id='170d799c17cf8d04',format="full").execute()
msg['snippet']

In [8]:
f=[" ".join(i.split()[:-1]) if len(i.split())>1 else i for i in ff]

In [9]:
import pandas as pd

In [10]:
df=pd.DataFrame(list(zip(id,date,ff,sub,msg_read,text)),columns=['Mail_Id','Date','From','Subject','Short Text','Body'])

In [12]:
df.head()

,Mail_Id,Date,From,Subject,Short Text,Body
0,171b70da99da5923,"Sun, 26 Apr 2020 15:14:14 +0000 (UTC)",UCR Graduate Business Admissions <agsmadmissio...,UC Riverside School of Business Virtual Info S...,UC Riverside School of Business Virtual Info S...,UCR School of Business link: www.agsm.ucr.edu ...
1,171b4e3ad02eb734,"Sun, 26 Apr 2020 05:01:46 +0000",Instaread <newsletters@instaread.co>,Your path to greatness,"Weekend Wisdom Hi there, A lot of companies ar...","html PUBLIC XHTML 1.0 t-family:arial, 'helveti..."
2,171b4ba361440057,"Sun, 26 Apr 2020 09:53:50 +0530 (IST)",<classes@naukrirms.com>,MNC Job Oriented Course ▷▷Data Analyst ▷▷Free ...,STRUCTURED LEARNING ASSISTANCE CONSULTANTS IND...,"html PUBLIC XHTML 1.0 class=3D""oh"" id=3D""mailL..."
3,171b4b75fb01bf35,"Sun, 26 Apr 2020 09:50:42 +0530 (IST)",<classes@naukrirms.com>,MNC Job Oriented Course ▷▷Data Analyst ▷▷Free ...,STRUCTURED LEARNING ASSISTANCE CONSULTANTS IND...,"html PUBLIC XHTML 1.0 class=3D""oh"" id=3D""mailL..."
4,171b4b02015f93ed,"Sun, 26 Apr 2020 09:42:48 +0530 (IST)",<classes@naukrirms.com>,MNC Job Oriented Course ▷▷Data Analyst ▷▷Free ...,STRUCTURED LEARNING ASSISTANCE CONSULTANTS IND...,"html PUBLIC XHTML 1.0 class=3D""oh"" id=3D""mailL..."


In [19]:
df_spam=df.iloc[:62]
df_spam.tail()

,Mail_Id,Date,From,Subject,Short Text,Body
57,171935c75e45a459,"Sun, 19 Apr 2020 08:33:27 -0400",Robert Kiyosaki <robertkiyosaki@fivemoneyrules...,Earning more income in this failing economy...,lessons from the 2008 CRASH - OPEN NOW ‌ ‌ ‌ ‌...,no ...I've been warning of something like this...
58,17192cbf5c979684,"Sun, 19 Apr 2020 14:16:10 +0000 (UTC)",UCR Graduate Business Admissions <agsmadmissio...,UC Riverside School of Business Virtual Info S...,UC Riverside School of Business Virtual Info S...,UCR School of Business link: www.agsm.ucr.edu ...
59,17192a665e42dc7b,"Sun, 19 Apr 2020 19:05:10 +0530",Swiggy <no-reply@swiggy.in>,Is your food safe? It is in these hands 👇,Don&#39;t want to receive updates from Swiggy?...,Email Template Don't want to receive updates f...
60,171924e295c88642,"Sun, 19 Apr 2020 09:43:01 +0000",Pranab Mishra <purabm@mails.stocknote.com>,Sharpen your stock market knowledge with knowl...,Open an Account Whether you are new to trading...,null
61,17191b41cbda144c,"Sun, 19 Apr 2020 14:40:23 +0530 (IST)",<kajal@naukrirms.com>,MNC Job Oriented Course ▷▷Data Analyst ▷▷Free ...,STRUCTURED LEARNING ASSISTANCE CONSULTANTS IND...,"html PUBLIC XHTML 1.0 class=3D""oh"" id=3D""mailL..."


In [21]:
df_spam.to_csv(r'C:\Users\lenovo\Documents\Project\my mail spam add.csv',index=None)

In [11]:
df.to_csv(r'C:\Users\lenovo\Documents\Project\my mail spam.csv',index=None)

In [ ]:
df.to_excel(r'C:\Users\lenovo\Documents\Project\my mail spam.xlsx',index=None)

In [ ]:
from __future__ import print_function
import pickle
import os.path
import base64
import email
from apiclient import errors
import re
import time
from datetime import datetime
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://mail.google.com/']

def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    
    return service

#     # Call the Gmail API
#     results = service.users().labels().list(userId='me').execute()
#     labels = results.get('labels', [])
    
#     userInfo = service.users().getProfile(userId='me').execute()
#     print("User Info :",userInfo)

#     if not labels:
#         print('No labels found.')
#     else:
#         print('Labels:')
#         for label in labels:
#             print(label['name'])





# if __name__ == '__main__':
#     main()

main()

In [ ]:
a='1719045aa8141abb'

In [ ]:
try:
    main().users().messages().delete(userId='me',id=a).execute()
    print('Message with id: %s deleted successfully.',a)
except errors.HttpError as error:
    print(error)